In [1]:
import pypsa
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

C:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
C:\Users\u6352049\AppData\Roaming\Python\Python39\site-packages\pypsa\networkclustering.py:16: UserWarning:

The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 



In [2]:
nodes = pd.read_csv('DATA - Substation Coordinates.csv')
edges = pd.read_csv('DATA - TRANSMISSION_LINES.csv', usecols=['OBJECTID', 'NAME', 'CAPACITY_KV', 'SHAPE_Length', 'OPERATIONAL_STATUS'])
nodes.drop(['OPERATIONAL_STATUS', 'ATTRIBUTE_SOURCE',
            'CUSTODIAN_AGENCY', 'CUSTODIAN_LICENSING'], axis=1)

# spare_edges = edges[edges['NAME'].str.contains(' to ')]

edges = edges[edges.OPERATIONAL_STATUS == 'Operational']
spare_edges = edges[~edges.NAME.str.contains(' to ')]
edges = edges[edges.NAME.str.contains(' to ')]

In [3]:
edges.head()

,OBJECTID,NAME,OPERATIONAL_STATUS,CAPACITY_KV,SHAPE_Length
0,1,Wagga 330 to Australian Newsprint Mills,Operational,132,1.005043
1,2,Albury to Australian Newsprint Mills,Operational,132,0.098778
2,3,Jindera to Australian Newsprint Mills,Operational,132,0.115721
3,4,Wodonga Terminal to Jindera,Operational,330,0.422441
4,5,Hume to Wodonga Terminal,Operational,66,0.132856


In [4]:
edges[['f', 't']] = edges.NAME.str.split(' to ', expand=True)

In [5]:
fro = edges.f.unique()
print(fro)

['Wagga 330' 'Albury' 'Jindera' ... 'Yandi' 'Hope Downs 1'
 'Cape Lambert Terminal']


In [26]:
x = 0
for _, i in edges.iterrows():
    if i.f in nodes.NAME.to_list():
        x+=1
print('possible', len(fro), ' | actual', x)
# def find_in_name():
# def find_in_address():

possible 1013  | actual 1014


In [25]:
print(nodes[nodes.NAME=='Wagga 330'].NAME)
print(edges[edges.f =='Wagga 330'].f)
if 'Wagga 330' in nodes.NAME.to_list():
    print(' ok')

270    Wagga 330
Name: NAME, dtype: object
0      Wagga 330
153    Wagga 330
156    Wagga 330
202    Wagga 330
211    Wagga 330
229    Wagga 330
408    Wagga 330
Name: f, dtype: object
 ok


In [9]:
mask = edges.f.isin(nodes.NAME)
mask1 = edges.t.isin(nodes.NAME)

In [10]:
print(sum(mask))
print(sum(mask1))

1359
1315


In [11]:
print(len(mask))
print(len(mask1))

2435
2435


In [12]:
mask2 = zip(mask,mask1)

In [13]:
x = [1 for i in mask2 if i[0]==True and i[1]==True]

In [14]:
print(len(x))

1036


In [37]:
buses = np.unique(list(set(edges.f.to_list() + edges.t.to_list())))
edges.drop_duplicates(['NAME'],inplace=True)
network = pypsa.Network()
for i in buses:
    if i in nodes.NAME.to_list():
        y = nodes[nodes.NAME==i].LATITUDE.to_list()[0]
        x = nodes[nodes.NAME==i].LONGITUDE.to_list()[0]
        network.add(
        "Bus",
        "{}".format(i),
        x=x, y=y
        )
    # else:
    #     x = 0
    #     y = 0

    # network.add(
    #     "Bus",
    #     "{}".format(i),
    #     x=x, y=y
    #     )

for index, row in edges.iterrows():
    network.add(
        "Line",
        "{}".format(row["NAME"]),
        bus0=row["f"],
        bus1=row["t"],
        # p_nom=1,
    )
# network.iplot(layouter=nx.spring_layout)
network.iplot(mapbox=True)

{'data': [Scattermapbox({
      'lat': [-35.20043347, -35.9941497],
      'line': {'color': 'rosybrown', 'width': 3},
      'lon': [147.3935504, 146.9846908],
      'mode': 'lines',
      'opacity': 0.8
  }),
  Scattermapbox({
      'lat': [-36.0538175, -35.9941497],
      'line': {'color': 'rosybrown', 'width': 3},
      'lon': [146.952647, 146.9846908],
      'mode': 'lines',
      'opacity': 0.8
  }),
  Scattermapbox({
      'lat': [-35.9253465, -35.9941497],
      'line': {'color': 'rosybrown', 'width': 3},
      'lon': [146.908546, 146.9846908],
      'mode': 'lines',
      'opacity': 0.8
  }),
  Scattermapbox({
      'lat': [nan, -35.9253465],
      'line': {'color': 'rosybrown', 'width': 3},
      'lon': [nan, 146.908546],
      'mode': 'lines',
      'opacity': 0.8
  }),
  Scattermapbox({
      'lat': [-36.10540959, nan],
      'line': {'color': 'rosybrown', 'width': 3},
      'lon': [147.0325562, nan],
      'mode': 'lines',
      'opacity': 0.8
  }),
  Scattermapbox({
      '

In [14]:
network = pypsa.Network()
nodes.drop_duplicates(['NAME'], inplace=True)
for index, row in nodes.iterrows():
    network.add(
        "Bus",
        "{}".format(row["NAME"]),
        x=row['LONGITUDE'],
        y=row['LATITUDE']
    )
network.iplot()

{'data': [{'x': Bus
   Clements Gap Wind Farm                138.130708
   Snowtown Windfarm South               138.118444
   Blyth West                            138.407543
   Belalie-North Brown Hill Wind Farm    138.727294
   Redbank                               151.072531
                                            ...    
   Fairview                              149.079454
   Gas processing facility               149.021518
   Jimboomba                             153.044262
   Jardine Creek                         144.412969
   Cape River East                       145.468356
   Name: x, Length: 841, dtype: float64,
   'y': Bus
   Clements Gap Wind Farm               -33.508265
   Snowtown Windfarm South              -33.830164
   Blyth West                           -33.846902
   Belalie-North Brown Hill Wind Farm   -33.309212
   Redbank                              -32.579544
                                           ...    
   Fairview                             -25.73078